# Performance Eval: fortran, numba, numpy

In [ ]:
import pathlib as pl
import pywatershed

repo_root = pywatershed.constants.__pywatershed_root__.parent
data_dir = pl.Path("../../../data/")

In [ ]:
# Instantiate control, params, inputs, model and run it to completion with budget choice
def proc_model_performance(process, domain, calc_method, budget_type: str = 'warn'):
    
    domain_dir = repo_root / f"test_data/{domain}"

    input_dir = domain_dir / "output"
    
    params = pywatershed.PrmsParameters.load(domain_dir / "myparam.param")
    control = pywatershed.Control.load(domain_dir / "control.test", params=params)

    input_variables = {}
    for key in process.get_inputs():
      nc_path = input_dir / f"{key}.nc"
      input_variables[key] = nc_path

    proc_model = process(
      control,
      **input_variables,
      budget_type=budget_type,
      calc_method=calc_method,
    )

    for istep in range(control.n_times):
      control.advance()
      proc_model.advance()
      proc_model.calculate(float(istep))

    proc_model.finalize()

    return

# Generate performance data

In [ ]:
domains = ['conus_2yr', 'drb_2yr', 'ucb_2yr', 'hru_1']
calc_methods = ['numba', 'fortran', 'numpy']
processes = [pywatershed.PRMSCanopy, pywatershed.PRMSChannel, pywatershed.PRMSGroundwater,]
results = []
ii = 0
for pp in processes:
    for dd in domains:
        for cc in calc_methods:
            
            if (dd == 'conus_2yr') and (pp == pywatershed.PRMSChannel):
                if (cc == 'fortran') or (cc == 'numpy'):
                    # some trouble with the networkx ordering? self._segment_order is wrong length going to f90
                    continue

            print(ii)
            ii += 1
            if (pp.__name__ != "PRMSGroundwater") and (cc == 'jax'):
                continue  # only implemented for PRMSGroundwater so far
            print('\n', pp.__name__, dd, cc)
            result = %timeit -o -n4 -r1 proc_model_performance(pp, dd, cc)
            results += [{(pp.__name__, dd, cc): result}]

In [ ]:
results_post = {}
for rr in results:
    kk = list(rr.keys())[0]
    vv = list(rr.values())[0]
    results_post[kk] = {'mean': vv.average, 'stdev': vv.stdev, 'N': vv.repeat}

In [ ]:
results_post

In [ ]:
# eventually save to disk using pickle
# this code worked well in pynhm_nhm_performance, do similar change up the path
# import pickle
# for path, result in results.items():
#     path = pl.Path(path)
#     pkl_path = path.parent.parent /  (f"results/{path.parent.name}_{path.name}_compile_performance.pkl")
#     print(pkl_path)
#
#     with open(pkl_path, "wb") as output_file:
#         pickle.dump(result, output_file)
# results2 = {}
# files = (data_dir / 'pynhm/performance_runs/results/').glob('*.pkl')
# for ff in files: 
#     print(ff)
#     with open(ff, "rb") as input_file:
#         results2[ff.name[0:-4]] = pickle.load(input_file)

In [ ]:
results_post_sav = results_post.copy()

In [ ]:
import pandas as pd
pd.options.plotting.backend = 'holoviews'

In [ ]:
results_df = pd.DataFrame(results_post).T
results_df.index.set_names(names = ["process", "domain", "calc"], inplace=True)
#results_df.sort_index(inplace=True) #this kills the show
categories_order = ['hru_1', 'drb_2yr', 'ucb_2yr', 'conus_2yr']
categories = pd.CategoricalIndex(results_df.index.levels[1].values,
                                 categories=categories_order,
                                 ordered=True)
results_df.index.set_levels(categories, level='domain', inplace=True)
results_df.sort_index(inplace=True)
results_df
# drop suspicious conus channel result
results_df.drop(('PRMSChannel', 'conus_2yr'), axis=0, inplace=True)

In [ ]:
for pp in ["PRMSGroundwater", 'PRMSCanopy', "PRMSChannel"]:
    proc_df = results_df.loc[pp, slice(None), slice(None), slice(None)] 
    display(
        proc_df.plot.bar().opts(
            title=pp, 
            height=450, width=800,
            ylabel='Mean Time (seconds)', 
            xlabel='',  # 'Domain: Calculation Method',
            xrotation = 65,
            fontscale=1.5,
            #ylim=ylim,
            show_grid=True,
            logy=True, 
        )
    )

In [ ]:
(proc_df.plot.bar().opts(title=pp) * 
 proc_df.hvplot.errorbars(y='mean', yerr1='stdev'))

In [ ]:
proc_df_ri = proc_df.reset_index()
proc_df_ri

In [ ]:
proc_df_ri.plot.bar(y='mean') * proc_df_ri.hvplot.errorbars(x='index',y='mean', yerr1='stdev')

# Performance profiling


In [ ]:
%load_ext snakeviz

In [ ]:
%%snakeviz
proc_model_performance(pywatershed.PRMSChannel, 'ucb_2yr', 'numpy')

In [ ]:
%%snakeviz
proc_model_performance(pywatershed.PRMSChannel, 'ucb_2yr', 'fortran')

## Notes on profiles

### PRMSGroundwater  
The calculations take about 2-2.5% of runtime regardless of method ?and domain?
The overall run times have a significant portion in reading input: storageUnit: advanceInput takes about 75% of runtime.  


### PRMSCanopy
hru_1: the calculations take about 1% of runtime
ucb_2yr: numpy calculations take about 75% of run time, fortran is about 2%. 

### PRMSChannel
hru_1: 
ucb_2_yr: numpy calculations take 95% of runtime. fortran calculations take 52% of runtime.